<a href="https://colab.research.google.com/github/mariprati/notebooks/blob/main/prova2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dicom2nifti
!pip install nibabel
!pip install totalsegmentator
!pip install shutil
!pip install pydicom
!pip install matplotlib
!pip install pandas
!pip install dcm2niix
! pip install subprocess
!pip install os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 102.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.0/197.0 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.5/347.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.

ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.4/451.4 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dcm2niix: filename=dcm2niix-1.0.20220715-cp310-cp310-linux_x86_64.whl size=559550 sha256=1b2bb4481aad3632193deacdec96f11ed3a02df803477d215ef69ab98aeadf51
  Stored in directory: /root/.cache/pip/wheels/88/8d/9b/5ea20c0451a1acddef585757be7dfec121ee076e58503b267c
Successfully built dcm2niix
ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
import os
import dicom2nifti
import nibabel as nib
from totalsegmentator.python_api import totalsegmentator
import shutil
import pydicom
import numpy as np
import dicom2nifti
import matplotlib.pyplot as plt
import pandas as pd
import dcm2niix
import subprocess


In [3]:
def dicom_folder_to_nifti(dicom_folder, output_folder):
    """
    Converts a directory of DICOM files to a NIfTI volume and saves it using dcm2niix.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Define the output filename format
    folder_name = os.path.basename(os.path.normpath(dicom_folder))
    filename_format = f"{folder_name}_nifti"

    # Construct the dcm2niix command
    command = [
        "dcm2niix",
        "-z", "y",  # Compress the output using gzip
        "-f", filename_format,  # Custom filename format
        "-o", output_folder,  # Output directory
        dicom_folder  # Input directory containing DICOM files
    ]

    try:
        # Execute the command
        result = subprocess.run(command, check=True, text=True, capture_output=True)
        print("DICOM to NIfTI conversion successful!")
        print("Output:", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error during conversion!")
        print("Output:", e.stderr)



def get_nifti_info(nifti_path):
    """Extracts and prints the dimensions and metadata of a NIfTI file."""
    # Load the NIfTI image
    nifti_img = nib.load(nifti_path)

    # Get the data array dimensions
    dimensions = nifti_img.shape  # This gives the number of pixels along each axis

    # Get the voxel size
    voxel_sizes = nifti_img.header.get_zooms()  # Pixel size along each axis in millimeters

    # Print the information
    print(f"Dimensions (pixels along each axis): {dimensions}")
    print(f"Voxel sizes (mm per axis): {voxel_sizes}")
    print(f"Data type: {nifti_img.get_data_dtype()}")
    print(f"Affine transformation matrix:\n{nifti_img.affine}")

    return {
        "dimensions": dimensions,
        "voxel_sizes": voxel_sizes,
        "data_type": nifti_img.get_data_dtype(),
        "affine": nifti_img.affine
    }

def find_and_save_slices(dicom_folder, target_value):
    """
    Finds the DICOM slice with a specific InstanceNumber, extracts its
    ImagePatientPosition, and returns it as a vector (x, y, z).

    Parameters:
        dicom_folder (str): Path to the folder containing DICOM files.
        target_value (int): The target InstanceNumber to find.

    Returns:
        tuple: The ImagePatientPosition as a tuple (x, y, z).
    """
    # Load all DICOM files in the folder and sort by Instance Number
    dicom_files = sorted(
        [f for f in os.listdir(dicom_folder) if f.endswith(".dcm")],
        key=lambda x: pydicom.dcmread(os.path.join(dicom_folder, x)).InstanceNumber
    )

    target_index = None
    img_patient_position = None

    # Iterate over DICOM files to find the slice with InstanceNumber equal to target_value
    for i, filename in enumerate(dicom_files):
        filepath = os.path.join(dicom_folder, filename)
        dicom_data = pydicom.dcmread(filepath)

        # Check if the InstanceNumber matches the target value
        if dicom_data.InstanceNumber == target_value:
            # Extract the ImagePatientPosition of this slice
            img_patient_position = dicom_data.ImagePositionPatient  # DICOM tag (0020,0032)
            target_index = i
            break

    # If target slice is not found, exit
    if target_index is None:
        print(f"No slice found with InstanceNumber equal to {target_value}")
        return None

    # If the slice is found, return the ImagePatientPosition as a tuple
    x, y, z = img_patient_position
    print(f"Slice with InstanceNumber {target_value} found at index {target_index}.")
    print(f"ImagePatientPosition: x={x}, y={y}, z={z}")
    return x, y, z

def world_to_voxel_coordinates(header, x, y, z):
    """
    Convert world coordinates (x, y, z) to voxel indices (i, j, k) using the NIfTI header.

    Parameters:
    header: NIfTI header object (e.g., from nibabel)
    x, y, z: World coordinates (float)

    Returns:
    i, j, k: Voxel indices (int)
    """
    # Extract necessary fields from the header
    pixdim = header["pixdim"]
    qoffset_x = header["qoffset_x"]
    qoffset_y = header["qoffset_y"]
    qoffset_z = header["qoffset_z"]
    qfac = pixdim[0]  # Either 1 or -1

    # Define the fixed rotation matrix R
    R = np.array([
        [1, 0,  0],
        [0, -1, 0],
        [0, 0, -1]
    ])

    # Subtract offsets to account for translation
    world_vector = np.array([x, y, z]) - np.array([qoffset_x, qoffset_y, qoffset_z])

    # Invert the rotation matrix (which is simple since it's a diagonal matrix)
    R_inv = np.linalg.inv(R)

    # Apply the inverse rotation
    voxel_vector = R_inv @ world_vector

    # Divide by the scaling factors to get the voxel indices
    i = voxel_vector[0] / pixdim[1]
    j = voxel_vector[1] / pixdim[2]
    k = voxel_vector[2] / (qfac * pixdim[3])

    # Round to the nearest integer to get the voxel indices
    i = round(i)
    j = round(j)
    k = round(k)

    return i, j, k

def find_sct_slice_number(csv_file, pid, study_yr):
    """
    Finds the sct_slice_number for the given pid and study_yr from a CSV file.

    Args:
        csv_file (str): Path to the CSV file.
        pid (str): Patient ID.
        study_yr (str): Study year.

    Returns:
        int: The corresponding sct_slice_number.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Filter the DataFrame for the specific pid and study_yr
    row = df[(df['pid'] == pid) & (df['study_yr'] == study_yr)]

    # Check if the row exists and retrieve sct_slice_number
    if not row.empty:
        print(f"Matching row from CSV:\n{row}\n")
        return int(row['sct_slice_num'].values[0])
    else:
        raise ValueError(f"No matching entry found for pid={pid} and study_yr={study_yr}")


def find_sct_epi_loc(csv_file, pid, study_yr):
    """
    Finds the sct_epi_loc for the given pid and study_yr from a CSV file.

    Args:
        csv_file (str): Path to the CSV file.
        pid (str): Patient ID.
        study_yr (str): Study year.

    Returns:
        int: The corresponding sct_epi_loc.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Filter the DataFrame for the specific pid and study_yr
    row = df[(df['pid'] == pid) & (df['study_yr'] == study_yr)]

    # Check if the row exists and retrieve sct_epi_loc
    if not row.empty:
        sct_epi_loc = int(row['sct_epi_loc'].values[0])
        print(f"Matching row from CSV:\n{row}\n")
        print(f"sct_epi_loc value: {sct_epi_loc}")
        return sct_epi_loc
    else:
        raise ValueError(f"No matching entry found for pid={pid} and study_yr={study_yr}")


def load_lung_lobe_nifti(masked_output_dir, folder_name, sct_epi_loc):
    """
    Loads the NIfTI file corresponding to the lobe based on sct_epi_loc and returns the region name.

    Args:
        masked_output_dir (str): Directory containing the masked NIfTI files.
        folder_name (str): Name of the folder containing the DICOM data.
        sct_epi_loc (int): Value determining which lobe to load.

    Returns:
        tuple: A tuple containing:
            - nib.Nifti1Image: The loaded NIfTI file.
            - str: The name of the region (e.g., 'lung_upper_lobe_right').
    """
    nifti_file = None
    region = None

    # Load the appropriate file based on sct_epi_loc
    if sct_epi_loc == 1:
        region = 'lung_upper_lobe_right'
        nifti_file = nib.load(os.path.join(masked_output_dir, f'{region}.nii.gz'))
        print("Loaded Right Upper Lobe")
    elif sct_epi_loc == 2:
        region = 'lung_middle_lobe_right'
        nifti_file = nib.load(os.path.join(masked_output_dir, f'{region}.nii.gz'))
        print("Loaded Right Middle Lobe")
    elif sct_epi_loc == 3:
        region = 'lung_lower_lobe_right'
        nifti_file = nib.load(os.path.join(masked_output_dir, f'{region}.nii.gz'))
        print("Loaded Right Lower Lobe")
    elif sct_epi_loc == 4:
        region = 'lung_upper_lobe_left'
        nifti_file = nib.load(os.path.join(masked_output_dir, f'{region}.nii.gz'))
        print("Loaded Left Upper Lobe")
    elif sct_epi_loc in [5, 6]:  # Lingula (5) treated as part of Left Lower Lobe (6)
        region = 'lung_lower_lobe_left'
        nifti_file = nib.load(os.path.join(masked_output_dir, f'{region}.nii.gz'))
        print("Loaded Left Lower Lobe (including Lingula)")
    else:
        raise ValueError(f"Invalid sct_epi_loc value: {sct_epi_loc}")

    return nifti_file, region


In [4]:
# %% MAIN

# La funzione *main* è divisa in tre parti:
# 1) Creazione del nifti volume a partire dai dicom files + Segmentazione del nifti volume
#     1. Carico la cartella contenente tutti i file *.dcm
#     2. Trasformazione de dicom a nifti
#     3. Definizione delle aree entro cui segmentare i polmoni:
#         'lung_upper_lobe_left',
#         'lung_lower_lobe_left',
#         'lung_upper_lobe_right',
#         'lung_middle_lobe_right',
#         'lung_lower_lobe_right'
#     4. Segmentazione attraverso *TotalSegmentator*
# 2) Applicazione maschera alla segmentazione in modo da avere valori discreti dell'immagine originale (+bkgrn messo a valore HU = -1000)
# 3) Estrazione del sottovolume (3 slices = 6cm) attorno alla slice centrale in cui il nodulo ha diametro maggiore
#     1. Definizione dei dati del paziente: *pid* e *study_yr*
#     2. Estrazione delle informazioni dal file *.csv* data la coppia *pid*-*study_yr*
#         1. **sci_epi_loc** --> zona dei polmoni in cui si trova la lesione, estrarrò il sottovolume della sua segmentazione
#         2. **sct_slice_num** --> tag (0020,0013), *InstanceNumber*, è la slice interessata dal diametro maggiore del nodulo
#     3. Salvataggio della variabile *ImagePatientPosition* della slice ricavata al punto precedente. Utile per la ricostruzione spaziale della slice interessata nel nifti volume.
#     4. Trasformazione affine e ottenimento di 3 slice del nifti volume.

In [10]:
def main():
    # %% Set environment variables at the start of the script
    os.environ['LC_ALL'] = 'C.UTF-8'
    os.environ['LANG'] = 'C.UTF-8'
    os.environ['LANGUAGE'] = 'C.UTF-8'

    # --------------------------------------------------------------------------------------------
    # %% SEGMENTAZIONE
    # --------------------------------------------------------------------------------------------
    #dicom_folder = '/Users/mariaprati/Desktop/100334_t1'  # Path to DICOM folder
    dicom_folder = '/content/100334_t1'
    output_folder = '/content/risultati'  # Path to output results

    # Extract the last part of the dicom_folder path (the folder name)
    folder_name = os.path.basename(os.path.normpath(dicom_folder))

    # Create a new output folder path inside Segmentazione2
    output_folder = os.path.join(output_folder, folder_name)

    # Ensure the new output folder exists
    os.makedirs(output_folder, exist_ok=True)
    dicom_folder_to_nifti(dicom_folder, output_folder)

    # Define paths for the segmentation
    #input_image_path = os.path.join(output_folder, f"{folder_name}_nifti_volume.nii")
    input_image_path = os.path.join(output_folder, f"{folder_name}_nifti.nii.gz")

    output_dir = os.path.join(output_folder, 'Lung_segmentations')

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Define the lung lobe names
    lobe_names = [
        'lung_upper_lobe_left',
        'lung_lower_lobe_left',
        'lung_upper_lobe_right',
        'lung_middle_lobe_right',
        'lung_lower_lobe_right'
    ]
    labels = {
        'lung_upper_lobe_left': 1,
        'lung_lower_lobe_left': 2,
        'lung_upper_lobe_right': 3,
        'lung_middle_lobe_right': 4,
        'lung_lower_lobe_right': 5
    }

    # Segment the lung lobes using TotalSegmentator
    totalsegmentator(
        input=input_image_path,
        output=output_dir,
        #fast=True,
        device="cpu",
        task="total",
        roi_subset=lobe_names
    )

    # Load the original CT image
    ct_img = nib.load(input_image_path)
    ct_data = ct_img.get_fdata()

    # Output directory for masked images
    masked_output_dir = os.path.join(output_dir, 'Lung_segmented_masks')
    os.makedirs(masked_output_dir, exist_ok=True)

    # Create and save masked images for each region
    for region, label in labels.items():
        seg_path = os.path.join(output_dir, f'{region}.nii.gz')
        if os.path.exists(seg_path):
            # Load the specific segmentation file for the current region
            seg_img = nib.load(seg_path)
            seg_data = seg_img.get_fdata()

            # Create a mask for the current region
            mask = seg_data == 1  # Assuming the segmentation file has binary mask (1 for region, 0 for background)

            # Apply the mask to the original CT data, setting background to -1000
            masked_ct_data = np.where(mask, ct_data, -1000)

            # Create a new NIfTI image for the masked data
            masked_ct_img = nib.Nifti1Image(masked_ct_data, ct_img.affine, ct_img.header)

            # Save the masked image
            #output_path = os.path.join(masked_output_dir, f'{region}_{folder_name}.nii.gz')
            output_path = os.path.join(masked_output_dir, f'{region}.nii.gz')

            nib.save(masked_ct_img, output_path)
        else:
            print(f"Segmentation file for {region}_{folder_name} not found at {seg_path}")

    print("Masked NIfTI files have been saved.")




    # --------------------------------------------------------------------------------------------
    # %% SUBVOLUME
    # --------------------------------------------------------------------------------------------
    csv_file = "/content/output.csv"  # Path to your CSV file
    #csv_file = "/Users/mariaprati/Desktop/filter_51_total.csv"  # Path to your CSV file
    #csv_file = "/Users/mariaprati/Desktop/filter_51_total_unique.csv"  # Path to your CSV file

    pid = 100334
    study_yr = 1

    # Load the CSV into a DataFrame
    df = pd.read_csv(csv_file)
    # Remove duplicates based on pid, study_yr, sct_epi_loc, and sct_slice_num
    #df = df.drop_duplicates(subset=['pid', 'study_yr', 'sct_epi_loc', 'sct_slice_num']) #ho commentato qui

    # Filter rows for the specified pid and study_yr
    rows = df[(df['pid'] == pid) & (df['study_yr'] == study_yr)]

    if rows.empty:
        raise ValueError(f"No entries found for pid={pid} and study_yr={study_yr}")

    print(f"Found {len(rows)} rows for pid={pid} and study_yr={study_yr}")

    # Process each row individually
    for index, row in rows.iterrows():
        sct_epi_loc = int(row['sct_epi_loc'])
        sct_slice_number = int(row['sct_slice_num'])

        # Load the lung lobe NIfTI file for the current row
        nifti_file, region = load_lung_lobe_nifti(masked_output_dir, folder_name, sct_epi_loc)

        print(f"Loaded NIfTI file: {nifti_file}")
        print(f"Corresponding region: {region}")

        header = nifti_file.header
        affine = nifti_file.affine
        data = nifti_file.get_fdata()
        pixdim = header["pixdim"]

        # Find the slice ImagePatientPosition
        result = find_and_save_slices(dicom_folder, sct_slice_number)

        if result:
            x, y, z = result
            print(f"Target slice ImagePatientPosition: x={x}, y={y}, z={z}")
        else:
            print("No slice found.")
            continue

        # Convert world coordinates to voxel indices
        i, j, k = world_to_voxel_coordinates(header, x, y, z)

        # Define the window size around the slice
        window_size = 1  # ±1 slices
        k_min = max(0, k - window_size)
        k_max = min(data.shape[2], k + window_size + 1)

        # Extract the sub-volume
        sub_volume = data[:, :, k]

        # Adjust the affine matrix for the sub-volume
        new_affine = affine.copy()
        new_affine[:3, 3] += k * pixdim[3]

        # Create a new NIfTI object for the sub-volume
        new_nifti = nib.Nifti1Image(sub_volume, affine=new_affine, header=header)

        # Save the sub-volume with a unique name
        output_path = os.path.join(
            masked_output_dir,
            f"lung_subvolume_pid{pid}_study{study_yr}_row{index}_{region}.nii.gz"
        )
        nib.save(new_nifti, output_path)
        print(f"Saved sub-volume for row {index} at: {output_path}")

    print("Processing complete.")
        ## aggiungo qui



        # --------------------------------------------------------------------------------------------
    # %% CLEANUP
    # --------------------------------------------------------------------------------------------
    # List all files in the output directory
    # all_files = os.listdir(masked_output_dir)

    # # Define the subvolume files to keep
    # files_to_keep = []
    # for index, row in rows.iterrows():
    #     sct_epi_loc = int(row['sct_epi_loc'])
    #     nifti_file, region = load_lung_lobe_nifti(masked_output_dir, folder_name, sct_epi_loc)
    #     files_to_keep.append(f"lung_subvolume_pid{pid}_study{study_yr}_row{index}_{region}.nii.gz")

    # # Delete all other .nii.gz files
    # for file_name in all_files:
    #     if file_name.endswith('.nii.gz') and file_name not in files_to_keep:
    #         file_path = os.path.join(masked_output_dir, file_name)
    #         os.remove(file_path)
    #         print(f"Deleted file: {file_path}")

    # print("Cleanup complete.")

    # # Delete the .nii.gz files in the 'Lung_segmentations' folder
    # segmentation_files = os.listdir(output_dir)
    # for file_name in segmentation_files:
    #     if file_name.endswith('.nii.gz'):
    #         file_path = os.path.join(output_dir, file_name)
    #         os.remove(file_path)
    #         print(f"Deleted segmentation file: {file_path}")

    # print("Segmentation cleanup complete.")

In [11]:
if __name__ == '__main__':
    main()

DICOM to NIfTI conversion successful!
Output: Chris Rorden's dcm2niiX version v1.0.20220505  GCC11.4.0 x86-64 (64-bit Linux)
Found 109 DICOM file(s)
Convert 109 DICOM as /content/risultati/100334_t1/100334_t1_nifti (512x512x109x1)
Conversion required 4.132330 seconds (4.108681 for core code).


If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 1.41s
Predicting...


/usr/local/lib/python3.10/dist-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(join(model_trai

  Predicted in 13.97s
Resampling...
  cropping from (512, 512, 109) to (452, 324, 109)
Resampling...
  Resampled in 2.55s
Predicting part 1 of 1 ...


/usr/local/lib/python3.10/dist-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(join(model_trai

  Predicted in 142.98s
Resampling...
Saving segmentations...


  0%|          | 0/5 [00:00<?, ?it/s]

  Saved in 5.75s
Masked NIfTI files have been saved.
Found 2 rows for pid=100334 and study_yr=1
Loaded Left Upper Lobe
Loaded NIfTI file: 
<class 'nibabel.nifti1.Nifti1Image'>
data shape (512, 512, 109)
affine:
[[  -0.66406202   -0.            0.          166.3999939 ]
 [   0.            0.66406202    0.         -190.63569641]
 [   0.            0.            2.5        -288.07998657]
 [   0.            0.            0.            1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 512 512 109   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.        0.664062  0.664062  2.5       0.        0.        0.
  0.      ]
vox_offset      : 0.0
scl_slope    